# Input data requirements

The input ice sheet model should be a netCDF file. 


### `Lithk` variable
The uploaded model to contain thickness data (the `lithk` variable) for the comparison.


In [1]:
import os,sys


# Get the current working directory
current_directory = os.getcwd()
# Navigate two levels up to reach main dir
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
# Insert the specific directory into the system path
sys.path.insert(0, os.path.join(parent_directory, 'bin', 'IMBIE'))
from imbie_utils import *

# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

warnings.filterwarnings('ignore')



### Configure IMBIE comparison

In [2]:
# Define the flag for the ice sheet region Greenland or Antarctica
icesheet = "AIS" # Change to "AIS" or "GIS"

# Define the flag for the area region Basin mask or Total area
area ="Basin" # Change to "Basin" or "Total"

# Set start and end dates
start_date = '2007-01-01'
end_date ='2009-01-01'

#Set density of ice
rho_ice = 918 # (kg/m^3)

#output file dirctory
output_path='/home/jovyan/CmCt/notebooks/IMBIE/'

# Select  variable for mass balance comparision
mass_balance_column="Cumulative mass balance (Gt)"# "Cumulative dynamics mass balance anomaly (Gt)"
if mass_balance_column == "Cumulative mass balance (Gt)":
    mass_balance_type = "total"
elif mass_balance_column == "Cumulative dynamics mass balance anomaly (Gt)":
    mass_balance_type = "dynamic"


# Set model model path,shapefile path and projection and IMBIE csv_file
if icesheet == "GIS":
    projection = "EPSG:3413"  # Greenland
    #Set the model data dir path
    # nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_JPL_ISSM_asmb.nc'
    nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_GSFC_ISSM_historical.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
    #Set the observation data dir path
    # obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2022_Gt_partitioned_v0.csv'
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2021_Gt.csv'

    ##Set the Region observation data dir path
    obs_east_filename = None
    obs_west_filename = None
    obs_peninsula_filename = None
    
    
elif icesheet== "AIS":
    projection = "EPSG:3031"  # Antarctica    
    #Set the model data dir path
    nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_AIS_AWI_PISM1_hist_std.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp"
    #Set the observation data dir path
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_antarctica_2022_Gt_partitioned_v0.csv'
    # obs_filename = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctica_2021_Gt.csv'
    
    ##Set the Region observation data dir path
    obs_east_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_east_antarctica_2022_Gt_partitioned_v0.csv'
    obs_west_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_west_antarctica_2022_Gt_partitioned_v0.csv'
    obs_peninsula_filename= '/home/jovyan/CmCt/data/IMBIE/imbie_antarctic_peninsula_2022_Gt_partitioned_v0.csv'

else:
    raise ValueError("Invalid iceshee value. Must be 'Greenland' or 'Antarctica'.")

In [3]:
# Check if  observation file exists
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# Check if model file exists    
if not os.path.exists(nc_filename):
    raise FileNotFoundError(f"Model file not found: {nc_filename}")


if icesheet== "AIS":   
    # Check if regional observation files exist 
    if not os.path.exists(obs_east_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
    if not os.path.exists(obs_west_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
    if not os.path.exists(obs_peninsula_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")

## Mass change comparision processing

In [4]:
# Calculate time-varying mass balance change of IMBIE from start to end date
# #IMBIE total mass balance
IMBIE_total_mass_change_sum=sum_MassBalance(obs_filename,start_date,end_date,mass_balance_column)


# Process the model data,calculate time-varying  model mass balance change from IMBIE's start to end date 
basin_result=process_model_data(nc_filename,IMBIE_total_mass_change_sum,start_date, end_date,rho_ice,projection,shape_filename,icesheet)

# Calculate difference of IMBIE-model mass change
results=process_IMBIE(obs_filename, start_date, end_date, icesheet, basin_result,IMBIE_total_mass_change_sum,mass_balance_column,obs_east_filename, obs_west_filename, obs_peninsula_filename)

The selected dates 2007-01-01 and 2009-01-01 are within the range of the model data.
Date 2007-01-31 not found in basin_result. Skipping.
Date 2007-01-31 not found in basin_result. Skipping.


## Display result and write it to csv file

In [5]:
# Extract the base name of the nc file (without .nc extension)
nc_base_filename = os.path.basename(nc_filename).replace('.nc', '')

# Create the CSV filename by combining the output path and the base nc filename with .csv extension
csv_filename = os.path.join(output_path, f"{nc_base_filename}.csv")

write_and_display_mass_change_comparison_all_dates(icesheet, basin_result, results, mass_balance_type, start_date, end_date, csv_filename)    # Initialize list to store rows of data for CSV
   


 Time-varying Mass change comparison (total): 2007-01-01 - 2009-01-01
Date            Basin/Region         Model mass change (Gt)    IMBIE mass change (Gt)    Residual (Gt)       
2007-01-01      Masked_Total         0.0                       0.0                       0.0                 
2007-03-02      Masked_Total         -7.00                     -38.64                    -31.65              
2007-04-02      Masked_Total         -13.99                    -57.96                    -43.97              
2007-05-02      Masked_Total         -20.76                    -77.28                    -56.52              
2007-06-02      Masked_Total         -27.76                    -96.61                    -68.85              
2007-07-02      Masked_Total         -34.53                    -115.93                   -81.40              
2007-08-01      Masked_Total         -41.30                    -135.25                   -93.95              
2007-09-01      Masked_Total         -48.29      

In [6]:
# ## Write the mass comaprision output results to csv files
# def write_and_display_mass_change_comparison_all_dates(icesheet,basin_result, results, mass_balance_type, start_date, end_date, csv_filename):
    
#     # Initialize list to store rows of data for CSV
#     data_rows = []

#     print_regionalresult_check = results.get('print_regionalresult_check')
    
#     # Add mass change comparison header
#     data_rows.append([f"Mass change comparison ({mass_balance_type})", f"{start_date} - {end_date}"])
#     data_rows.append(['Date', 'Basin/Region', 'Model mass change (Gt)', 'IMBIE mass change (Gt)', 'Residual (Gt)'])
    
#     print(f"\n Time-varying Mass change comparison ({mass_balance_type}): {start_date} - {end_date}")
#     print(f"{'Date':<15} {'Basin/Region':<20} {'Model mass change (Gt)':<25} {'IMBIE mass change (Gt)':<25} {'Residual (Gt)':<20}")
    
#     for date, result in results.items():
#         if date in basin_result:
#             # Basin mass change sums
#             basin_mass_change_sums = basin_result[date].get('basin_mass_change_sums', {})

            
#             for basin, model_mass_change in basin_mass_change_sums.items():
#                 imbie_mass_change = '--'
#                 residual_mass_change = '--'
#                 # print(f"{date:<15} {basin:<20} {model_mass_change:<25.2f} {imbie_mass_change:<25} {residual_mass_change:<20}")
#                 data_rows.append([date, basin, f"{model_mass_change:.2f}", imbie_mass_change, residual_mass_change])
     

#             if icesheet == "AIS" and print_regionalresult_check == 'YES':
#                 # Regional mass change sums
#                 region_mass_change_sums = basin_result[date].get('region_mass_change_sums', {})
#                 for region, model_mass_change in region_mass_change_sums.items():
#                     imbie_mass_change = results.get('Regional_Mass_Change_Summary', {}).get(date, {}).get(f'IMBIE_Mass_Change_{region}', 'N/A')
#                     residual_mass_change = results.get('Regional_Mass_Change_Summary', {}).get(date, {}).get(f'Delta_MassChange_{region}', 'N/A')
        
#                     imbie_mass_change = f"{imbie_mass_change:.2f}" if isinstance(imbie_mass_change, (float, int)) else "N/A"
#                     residual_mass_change = f"{residual_mass_change:.2f}" if isinstance(residual_mass_change, (float, int)) else "N/A"
#                     # print(f"{date:<15} {region:<20} {model_mass_change:<25.2f} {imbie_mass_change:<25} {residual_mass_change:<20}")
#                     # data_rows.append([date, region, f"{model_mass_change:.2f}", imbie_mass_change, residual_mass_change])
#                     data_rows.append([date, region, f"{model_mass_change:.2f}", imbie_mass_change, residual_mass_change])
            
#             # Total mass balance masked
#             model_total_mass_balance_masked = basin_result[date].get('model_total_mass_balance_masked', 'N/A')   
#             imbie_total_mass_change_sum = result.get('IMBIE_total_mass_change_sum', 'N/A')
#             delta_masschange_masked  = result.get('Delta_MassChange_masked', 'N/A')
    
#             model_total_mass_balance_masked  = f"{model_total_mass_balance_masked :.2f}" if isinstance(model_total_mass_balance_masked , (float, int)) else "N/A"
#             imbie_total_mass_change_sum = f"{imbie_total_mass_change_sum:.2f}" if isinstance(imbie_total_mass_change_sum, (float, int)) else "N/A"
#             delta_masschange_masked  = f"{delta_masschange_masked :.2f}" if isinstance(delta_masschange_masked , (float, int)) else "N/A"
#             print(f"{date:<15} {'Masked_Total':<20} {model_total_mass_balance_masked :<25} {imbie_total_mass_change_sum:<25} {delta_masschange_masked :<20}")
#             data_rows.append([date, 'Masked_Total', model_total_mass_balance_masked , imbie_total_mass_change_sum, delta_masschange_masked ])      
        

#             # Total mass balance unmasked
#             model_total_mass_balance_unmasked = basin_result[date].get('model_total_mass_balance_unmasked', 'N/A')
#             imbie_total_mass_change_sum = result.get('IMBIE_total_mass_change_sum', 'N/A')
#             delta_masschange_unmasked = result.get('Delta_MassChange_unmasked', 'N/A')
        
        
#             model_total_mass_balance_unmasked = f"{model_total_mass_balance_unmasked:.2f}" if isinstance(model_total_mass_balance_unmasked, (float, int)) else "N/A"
#             imbie_total_mass_change_sum = f"{imbie_total_mass_change_sum:.2f}" if isinstance(imbie_total_mass_change_sum, (float, int)) else "N/A"
#             delta_masschange_unmasked = f"{delta_masschange_unmasked:.2f}" if isinstance(delta_masschange_unmasked, (float, int)) else "N/A"

#             # print(f"{date:<15} {'Total':<20} {model_total_mass_balance_unmasked:<25} {imbie_total_mass_change_sum:<25} {delta_masschange_unmasked:<20}")
#             data_rows.append([date, 'Unmasked_Total', model_total_mass_balance_unmasked, imbie_total_mass_change_sum, delta_masschange_unmasked])

#    # Convert the data rows into a pandas DataFrame
#     df = pd.DataFrame(data_rows)
    
#     # Write the DataFrame to a CSV file
#     print(f"\nWriting data to CSV file: {csv_filename}")
#     df.to_csv(csv_filename, index=False, header=False)

In [7]:
IMBIE_total_mass_change_sum

,Year,Mass balance (Gt/yr),Mass balance uncertainty (Gt/yr),Cumulative mass balance (Gt),Cumulative dynamics mass balance anomaly uncertainty (Gt),Surface mass balance anomaly (Gt/yr),Surface mass balance anomaly uncertainty (Gt/yr),Cumulative surface mass balance anomaly (Gt),Cumulative surface mass balance anomaly uncertainty (Gt),Dynamics mass balance anomaly (Gt/yr),Dynamics mass balance anomaly uncertainty (Gt/yr),Cumulative dynamics mass balance anomaly (Gt),Cumulative dynamics mass balance anomaly uncertainty (Gt).1,Date,Month_Order,IMBIE_Mass_Change
337,2007-01-31,-231.8530,101.7379,-2208.9532,852.6869,-143.209106,37.627499,202.678772,879.555940,-88.643894,108.473172,-2411.632130,215.739682,2007-01-31 09:42:28.800,2,-19.3210
338,2007-03-02,-231.8530,101.7379,-2228.2743,853.1926,-143.209106,37.627499,190.744679,880.113167,-88.643894,108.473172,-2419.019121,216.012953,2007-03-02 20:17:31.200,3,-38.6421
339,2007-04-02,-231.8530,101.7379,-2247.5954,853.6979,-143.209106,37.627499,178.810587,880.670042,-88.643894,108.473172,-2426.406112,216.285880,2007-04-02 06:00:00.000,4,-57.9632
340,2007-05-02,-231.8530,101.7379,-2266.9165,854.2030,-143.209106,37.627499,166.876495,881.226565,-88.643894,108.473172,-2433.793103,216.558462,2007-05-02 15:42:28.800,5,-77.2843
341,2007-06-02,-231.8530,101.7379,-2286.2376,854.7077,-143.209106,37.627499,154.942403,881.782737,-88.643894,108.473172,-2441.180094,216.830702,2007-06-02 02:17:31.200,6,-96.6054
342,2007-07-02,-231.8530,101.7379,-2305.5586,855.2121,-143.209106,37.627499,143.008311,882.338558,-88.643894,108.473172,-2448.567086,217.102600,2007-07-02 12:00:00.000,7,-115.9264
343,2007-08-01,-231.8530,101.7379,-2324.8797,855.7163,-143.209106,37.627499,131.074218,882.894029,-88.643894,108.473172,-2455.954077,217.374158,2007-08-01 21:42:28.800,8,-135.2475
344,2007-09-01,-231.8530,101.7379,-2344.2008,856.2201,-143.209106,37.627499,119.140126,883.449151,-88.643894,108.473172,-2463.341068,217.645377,2007-09-01 08:17:31.200,9,-154.5686
345,2007-10-01,-231.8530,101.7379,-2363.5219,856.7237,-143.209106,37.627499,107.206034,884.003924,-88.643894,108.473172,-2470.728059,217.916259,2007-10-01 18:00:00.000,10,-173.8897
346,2007-11-01,-231.8530,101.7379,-2382.8430,857.2269,-143.209106,37.627499,95.271942,884.558349,-88.643894,108.473172,-2478.115050,218.186805,2007-11-01 03:42:28.800,11,-193.2108


In [8]:

### Extract time varying IMBIE mass balance data and calculate the time varying mass difference 


# Load the CSV file
mass_balance_data = pd.read_csv(obs_filename)

# Column names
date_column = 'Year'

# Ensure the 'Year' column is treated as float to capture the fractional year part
mass_balance_data['Year'] = mass_balance_data['Year'].astype(float)

# Apply the conversion function to the 'Year' column
mass_balance_data['Date'] = mass_balance_data['Year'].apply(fractional_year_to_date)

# Sort the data by 'Date' column to ensure it’s in increasing order of both year and fraction
mass_balance_data = mass_balance_data.sort_values(by='Date')
  
# Apply the function to each group of data (grouped by the year)
mass_balance_data = mass_balance_data.groupby(mass_balance_data['Date'].dt.year).apply(assign_month_order)


# Convert 'Year' column to year-month-day format where month is 'Month_Order'    
mass_balance_data['Year'] = mass_balance_data.apply(lambda row: f"{row['Date'].year}-{str(row['Date'].month).zfill(2)}-{str(row['Date'].day).zfill(2)}", axis=1) 
# mass_balance_data['Year'] = mass_balance_data.apply(lambda row: f"{row['Date'].year}-{str(row['Month_Order']).zfill(2)}-01", axis=1)    

# Reset the index to flatten the multi-index structure
mass_balance_data = mass_balance_data.reset_index(drop=True)


# Check if the column exists in the DataFrame
if mass_balance_column not in mass_balance_data.columns:
    raise ValueError(f"Error: The column '{mass_balance_column}' does not exist in the CSV file.")


# Get the initial mass balance value for the start date
data_start_date = mass_balance_data[mass_balance_data['Year'] == start_date]
if data_start_date.empty:
    raise ValueError(f"Error: No data available for the start date {start_date}.")
mass_balance_start_value = data_start_date[mass_balance_column].iloc[0]  # value of start date

# Filter data between start_date_converted and end_date_converted (inclusive)
filtered_data = mass_balance_data[
    (mass_balance_data['Year'] > start_date) & (mass_balance_data['Year'] <= end_date)
]


# Initialize the previous day's mass balance value to the starting mass balance
previous_mass_balance = mass_balance_start_value

# Calculate daily mass change from the previous day for each time step
mass_changes = []  # To store the daily mass changes

for index, row in filtered_data.iterrows():
    current_mass_balance = row[mass_balance_column]
    # Calculate the change from the previous day's balance
    mass_change = current_mass_balance-previous_mass_balance
    mass_changes.append(mass_change)


# Assign the calculated mass changes to a new column in the DataFrame
filtered_data['IMBIE_Mass_Change'] = mass_changes




In [10]:
filtered_data

,Year,Mass balance (Gt/yr),Mass balance uncertainty (Gt/yr),Cumulative mass balance (Gt),Cumulative dynamics mass balance anomaly uncertainty (Gt),Surface mass balance anomaly (Gt/yr),Surface mass balance anomaly uncertainty (Gt/yr),Cumulative surface mass balance anomaly (Gt),Cumulative surface mass balance anomaly uncertainty (Gt),Dynamics mass balance anomaly (Gt/yr),Dynamics mass balance anomaly uncertainty (Gt/yr),Cumulative dynamics mass balance anomaly (Gt),Cumulative dynamics mass balance anomaly uncertainty (Gt).1,Date,Month_Order,IMBIE_Mass_Change
337,2007-01-31,-231.8530,101.7379,-2208.9532,852.6869,-143.209106,37.627499,202.678772,879.555940,-88.643894,108.473172,-2411.632130,215.739682,2007-01-31 09:42:28.800,2,-19.3210
338,2007-03-02,-231.8530,101.7379,-2228.2743,853.1926,-143.209106,37.627499,190.744679,880.113167,-88.643894,108.473172,-2419.019121,216.012953,2007-03-02 20:17:31.200,3,-38.6421
339,2007-04-02,-231.8530,101.7379,-2247.5954,853.6979,-143.209106,37.627499,178.810587,880.670042,-88.643894,108.473172,-2426.406112,216.285880,2007-04-02 06:00:00.000,4,-57.9632
340,2007-05-02,-231.8530,101.7379,-2266.9165,854.2030,-143.209106,37.627499,166.876495,881.226565,-88.643894,108.473172,-2433.793103,216.558462,2007-05-02 15:42:28.800,5,-77.2843
341,2007-06-02,-231.8530,101.7379,-2286.2376,854.7077,-143.209106,37.627499,154.942403,881.782737,-88.643894,108.473172,-2441.180094,216.830702,2007-06-02 02:17:31.200,6,-96.6054
342,2007-07-02,-231.8530,101.7379,-2305.5586,855.2121,-143.209106,37.627499,143.008311,882.338558,-88.643894,108.473172,-2448.567086,217.102600,2007-07-02 12:00:00.000,7,-115.9264
343,2007-08-01,-231.8530,101.7379,-2324.8797,855.7163,-143.209106,37.627499,131.074218,882.894029,-88.643894,108.473172,-2455.954077,217.374158,2007-08-01 21:42:28.800,8,-135.2475
344,2007-09-01,-231.8530,101.7379,-2344.2008,856.2201,-143.209106,37.627499,119.140126,883.449151,-88.643894,108.473172,-2463.341068,217.645377,2007-09-01 08:17:31.200,9,-154.5686
345,2007-10-01,-231.8530,101.7379,-2363.5219,856.7237,-143.209106,37.627499,107.206034,884.003924,-88.643894,108.473172,-2470.728059,217.916259,2007-10-01 18:00:00.000,10,-173.8897
346,2007-11-01,-231.8530,101.7379,-2382.8430,857.2269,-143.209106,37.627499,95.271942,884.558349,-88.643894,108.473172,-2478.115050,218.186805,2007-11-01 03:42:28.800,11,-193.2108


In [11]:
mass_balance_start_value

-2189.6322

In [12]:
-2208.9532--2189.6322

-19.320999999999913

In [13]:
-2228.2743--2189.6322

-38.64210000000003